## 6-a. Embedding 계획
- 1. 토큰을 벡터화한다
  - 대표적으로 'Bag of Words', 'TF-IDF', 'Word2Vec' 의 방법이 있다
  - 여기서는 'TF-IDF' 를 사용해보려고 한다.
- 2. 패딩

## 6-b. 임베딩 진행 : TF-IDF 사용

### 6-b-1. 파일 열기

In [2]:
import pandas as pd
df = pd.read_csv("5_tokenized_text_10000.csv", encoding="utf-8")
df.tail(3)

,search_keyword,date_created,time_created,writer,is_reply,id,spaced_text,tokens,tokens_reduced
891011,휴젤,2020-04-29,11:55:10,ㅇㅇ(123.215),0,241400125127,미국주식보다 한국꺼보면 ㄹㅇ 암걸릴 것 같음 특히 휴젤 저거 아는분이 임원진인데 회...,"['미국', '주식', '보다', '한국', '꺼', '보', '면', 'ㄹㅇ', ...","['미국', '주식', '보다', '한국', '꺼', '보', '면', 'ㄹㅇ', ..."
891012,휴젤,2020-04-29,11:56:16,ㅇㄷ(175.223),1,241400125127,한국 상장회사들중에 ㅂㅅ같은곳이 넘 많음...,"['한국', '상장', '회사', '들', '중', '에', 'ㅂㅅ', '같', '...","['한국', '상장', '회사', '들', '중', '에', 'ㅂㅅ', '같', '..."
891013,휴젤,2020-04-29,11:56:38,ㅇㅇ(223.33),1,241400125127,신흥국 회사투자 망설여지는 게 이거 때문임. 주식시장에 대한 개념이 안 잡혀있음.,"['신흥국', '회사', '투자', '망설여', '지', '는', '게', '이거'...","['신흥국', '회사', '투자', '<UNK>', '지', '는', '게', '이..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891014 entries, 0 to 891013
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   search_keyword  891014 non-null  object
 1   date_created    891014 non-null  object
 2   time_created    891014 non-null  object
 3   writer          891014 non-null  object
 4   is_reply        891014 non-null  int64 
 5   id              891014 non-null  int64 
 6   spaced_text     891014 non-null  object
 7   tokens          891014 non-null  object
 8   tokens_reduced  891014 non-null  object
dtypes: int64(2), object(7)
memory usage: 61.2+ MB


### 6-b-2. 임베딩 

In [8]:
!pip install keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB 3.5 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.7 MB 5.6 MB/s eta 0:00:01
   ---------------------------- ----------- 1.2/1.7 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 7.2 MB/s eta 0:00:00


In [10]:
!pip install tensorflow

  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp38-cp38-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.25.1-cp38-cp38-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic 2.5.2 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.14.5 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [4]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding

# 토큰화 및 단어 사전 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tokens'])
sequences = tokenizer.texts_to_sequences(df['tokens'])

# 패딩 적용
max_seq_length = max(len(x) for x in sequences)
sequences_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

# 단어 사전 크기 및 임베딩 차원 설정
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

# 임베딩 층을 포함하는 모델 생성
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_seq_length))

# 모델 요약 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 776, 50)           5739400   
                                                                 
Total params: 5739400 (21.89 MB)
Trainable params: 5739400 (21.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 6-b-3. 임베딩 결과 파일로 저장

In [5]:
from keras.models import Sequential
from keras.layers import Embedding
# 모델 생성 및 컴파일 코드는 여기에 추가합니다.

# 모델 구조와 가중치를 파일로 저장
model.save('6_embedding_model.h5')

C:\Users\USER\.conda\envs\brnn\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### 6-b-4. 결과 확인 : 불러오기

In [6]:
from keras.models import load_model

# 모델 파일 로드
loaded_model = load_model('6_embedding_model.h5')

### 6-b-5. 단어 사전 크기, 임베딩 차원 확인하기

In [8]:
from keras.models import load_model

# 모델 파일 경로
model_file_path = '6_embedding_model.h5'

# 모델 로드
embedding_model = load_model(model_file_path)

# 임베딩 레이어의 가중치 확인
embedding_weights = embedding_model.layers[0].get_weights()[0]
embedding_weights.shape # 임베딩 차원과 단어 사전 크기 확인

(114788, 50)